# Constrained optimization problems



## what is quantum-classical hybrid?
Mix of classical and quantum compute resources working together on a problem

Take a look at: https://docs.ocean.dwavesys.com/en/stable/overview/hybrid.html

# Example:  Knapsack problem


The knapsack problem is a well-known optimization problem. It is encountered, for example, in packing shipping containers. 

A shipping container has a weight capacity which it can hold. Given a collection of items to be shipped, where each item has a value and a weight, the problem is to select the optimal items to pack in the shipping container. This optimization problem can be defined as an objective with a constraint:


    -Objective: Maximize freight value (sum of values of the selected items).
    -Constraint: Total freight weight (sum of weights of the selected items) must be less than or equal to the container's capacity.
    
<img src="files/knapsack.png" alt="" style="width: 500px;"/>

## In a nutshell: 

You have a knapsack with a weight limit — it can only carry a certain maximum weight. You have many items, each with a different weight, and a different monetary value. You want to choose a number of items to carry in the knapsack, while maximizing the monetary value of the whole transport, but without exceeding the maximum allowable weight.

In [5]:
#importing libraries

import pandas as pd
import numpy as np
import pyomo.environ as pe
from dimod.reference.samplers import ExactCQMSolver
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel
import time

In [6]:
#Generation of data
#randomly generated dataset containing 50 thousand items, 
#with weights and values varying randomly in a discrete uniform 
#distribution between 1 and 9999. The knapsack weight limit was set 
#to 80% of the total weight of all items — so some items will always be discarded.

n_items = 50000
max_cost = 9999
max_weight = 9999

rebuild_ds = True
if rebuild_ds:
    df = pd.DataFrame(
        {
            "cost": np.random.randint(1, max_cost, size=n_items),
            "weight": np.random.randint(1, max_weight, size=n_items),
        }
    )
    df.to_csv("ds.csv", index=False, header=False)

In [7]:
#CLASSICAL SOLVER (for later comparison)

#Just take it as it is :)

def knapsack_pyomo(values, weights, max_weight, debug=False):
    print(f"solving for {len(values)} items")
    print("build the model")
    pecm = pe.ConcreteModel(name="Knapsack")
    pecm.x = pe.Var(range(0, len(values)), domain=pe.Boolean)
    pecm.worth = pe.Objective(
        expr=sum(values[j] * pecm.x[j] for j in range(0, len(values))),
        sense=pe.maximize,
    )
    pecm.weight = pe.ConstraintList()
    pecm.weight.add(
        sum(weights[j] * pecm.x[j] for j in range(0, len(values))) <= max_weight
    )
    if debug:
        pecm.pprint()

    solver_name = "glpk"
    print(f"submit model to solver {solver_name}")
    solver = pe.SolverFactory(solver_name)
    t_start = time.time()
    solver.solve(pecm)
    t_end = time.time()
    t_solver = t_end - t_start
    if debug:
        pecm.display()

    print("parse the solver output")
    total_value = int(pecm.worth())
    total_weight = int(sum(weights[j] * pecm.x[j]() for j in range(0, len(values))))
    selected_items = pecm.x

    return (
        total_value,
        total_weight,
        selected_items,
        t_solver,
    )

## Hybrid solver

CQM cannot run directly on a quantum annealer. However, you can still code the problem using the D-Wave API and submit it to the solvers in the cloud. D-Wave will decompose the problem into BQM (which can be solved by the quantum machine) and the rest (which is then solved by D-Wave using regular solvers).

Think of it as divide-and-conquer: D-Wave tries to solve as much of your model as possible using the quantum annealer, but some problems still have parts that do not conform to the BQM equations. The combination of quantum and classical solvers that can solve different kinds of quadratic models is called a hybrid solver.

In [8]:
#Solving the problem with Dwave

def knapsack_dwave(values, weights, max_weight, debug=False, classic_solver=False):
    print(f"solving for {len(values)} items")
    print("build the model")
    
    #hint: cqm stands for constrained quadratic model, 
    #The model for this problem cannot be pure BQM, since we have a constraint
    cqm = ConstrainedQuadraticModel() 
    obj = BinaryQuadraticModel(vartype="BINARY")
    constraint = QuadraticModel()

    for i in range(len(values)):
        obj.add_variable(i)
        obj.set_linear(i, -values[i])
        constraint.add_variable("BINARY", i)
        constraint.set_linear(i, weights[i])

    cqm.set_objective(obj) #objective function
    cqm.add_constraint(constraint, sense="<=", rhs=max_weight, label="capacity") #constraints

   
    sampler = LeapHybridCQMSampler() #The model for this problem cannot be pure BQM, 
        #since we have a constraint. Therefore, we choose a hybrid sampler.


    print(f"submit model to solver {sampler.solver.name}")
    sampleset = sampler.sample_cqm(cqm, label="knapsack problem")

    #many sample sets may be returned by D-Wave as part of the solution. 
    #Some are not feasible (e.g. violate constraints) and we need to filter them out of the results. 
    #Out of the feasible sample sets, one or several will have the lowest Ising energy — 
    #those are the best solutions found by the quantum computer.    
    print("parse the solver output")
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
    if not len(feasible_sampleset):
        raise ValueError("No feasible solution found")
    best = feasible_sampleset.first

    selected_items = [key for key, val in best.sample.items() if val == 1.0]
    total_weight = sum(list(weights.loc[selected_items]))
    total_value = sum(list(values.loc[selected_items]))
    best_solution_energy = best.energy

  
    t_solver_server_side = sampleset.info["run_time"] / 1000000
    t_qpu = sampleset.info["qpu_access_time"] / 1000000

    return (
        total_value,
        total_weight,
        selected_items,
        t_solver_server_side,
        t_qpu,
        best_solution_energy,
        sampleset,
    )



# Results

In [13]:
ds = pd.read_csv("ds.csv", names=["cost", "weight"])

values_array = ds["cost"]
weights_array = ds["weight"]
knapsack_max_weight = int(0.8 * ds["weight"].sum()) #The knapsack weight limit was set 
#to 80% of the total weight of all items — so some items will always be discarded.


In [11]:
#compute the classical solution
finalVal, finalWeight, finalChoices, solver_time = knapsack_pyomo(
    values_array, weights_array, knapsack_max_weight
)

#printing the results
print("+++++++++++++++++++++++++++++++++")
print("**Classical solver's results**")
print(f"solver time:         {solver_time}")
print(f"knapsack max weight: {knapsack_max_weight}")
print(f"items total weight:  {finalWeight}")
print(f"items total value:   {finalVal}")

solving for 50000 items
build the model
submit model to solver glpk
parse the solver output
+++++++++++++++++++++++++++++++++
**Classical solver's results**
solver time:         20.075597286224365
knapsack max weight: 200829990
items total weight:  200829938
items total value:   242840637


GLPK took 25 seconds to solve the problem. 

It came up with a combination of items that is just under the maximum allowed knapsack weight.

In [22]:
#compute quantum solutions
(
    finalVal,
    finalWeight,
    finalChoices,
    solver_time_server,
    solver_time_qpu,
    best_energy,
    sampleset,
) = knapsack_dwave(values_array, weights_array, knapsack_max_weight)

#printing the results
print("+++++++++++++++++++++++++++++++++")
print(f"solver time server side: {solver_time_server}")
print(f"solver time QPU:         {solver_time_qpu}")
print(f"knapsack max weight:     {knapsack_max_weight}")
print(f"items total weight:      {finalWeight}")
print(f"items total value:       {finalVal}")
print(f"best sol. energy:        {best_energy}")

solving for 50000 items
build the model
submit model to solver hybrid_constrained_quadratic_model_version1p
parse the solver output
solver time server side: 14.512364
solver time QPU:         0.016045
knapsack max weight:     200604267
items total weight:      200604245
items total value:       241907898
best sol. energy:        -241907898.0


The total time spent by the D-Wave hybrid solver was around 12-14 seconds. This is an improvement over integer programming. Moreover, slight increases in the number of items force the integer programming code to spend a much, much longer time solving the problem — the increase is non-linear.

Not so with the D-Wave solver. It, too, spends a longer time solving a bigger problem, but the time it takes to do that does not increase catastrophically with the number of items.

The time spent on the actual QPU (the quantum annealer) is only 16 milliseconds. Most of the total time is spent decomposing the problem, sampling the solution space, then sending the results back to the coder. The larger the fraction of the problem that runs on the actual QPU, the bigger the speed gain.



References:

    1. https://github.com/dwave-examples/knapsack/blob/master/README.md
    
    2. https://towardsdatascience.com/quantum-computing-for-optimization-problems-solving-the-knapsack-problem-274f01e78ed8
